In [1]:
import os

import pandas as pd
import utils.dhs_config as dhs_config
import utils.dhs_utils as dhs

In [2]:
FILE_INDIV = "SOKA_SpDa_IndivData_toshare.csv"
FILE_DATES = "SOKA_SpDa_SpeedDateData_toshare.csv"

df_indiv = pd.read_csv(os.path.join(DATA, "raw", FILE_INDIV), index_col=0)
df_dates = pd.read_csv(os.path.join(DATA, "raw", FILE_DATES), index_col=0)

NameError: name 'DATA' is not defined

In [ ]:
df_dates

In [ ]:
retour = []
lst_couple = []

for i, row in df_dates.iterrows():
    targID = row["TargID"]
    percID = row["PercID"]
    couple = [targID, percID]
    couple.sort()
    couple = f"{couple[0]}, {couple[1]}"
    retour.append(
        df_dates[
            (df_dates.PercID == targID) & (df_dates.TargID == percID)
        ].FE_Match.values
    )
    lst_couple.append(couple)

df_dates["FE_Match_retour"] = np.array(retour).flatten()
df_dates["Match"] = df_dates.FE_Match_retour * df_dates.FE_Match
df_dates["Couple"] = lst_couple

In [ ]:
df_relation = df_dates.drop_duplicates("Couple")[["PercID", "TargID", "Match"]].dropna()

In [7]:
matrix_relation = (
    df_relation.pivot(index="PercID", columns="TargID", values="Match")
    .fillna(0)
    .astype("int")
)

In [8]:
matrix_relation

TargID,SD_1_01_W,SD_1_02_M,SD_1_02_W,SD_1_03_M,SD_1_03_W,SD_1_04_M,SD_1_04_W,SD_1_05_W,SD_1_06_M,SD_1_06_W,...,SD_7_08_M,SD_7_08_W,SD_7_09_M,SD_7_09_W,SD_7_10_M,SD_7_10_W,SD_7_11_W,SD_7_12_M,SD_7_12_W,SD_7_13_M
PercID,,,,,,,,,,,,,,,,,,,,,
SD_1_01_M,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SD_1_01_W,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SD_1_02_M,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SD_1_02_W,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SD_1_03_M,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SD_7_10_M,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
SD_7_10_W,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
SD_7_11_W,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [9]:
(matrix_relation.sum(axis=1) == 0).sum()

83

# DHS

In [3]:
# Main folder with respect from the scripts
MAIN = "../"

# Data folder
DATA = os.path.join(MAIN, "data")

# Raw data folder (where .sav files are)
RAW_DATA = os.path.join(DATA, "raw")

# Transformed data folder
TRS_DATA = os.path.join(DATA, "transformed")

column_variables, column_others = dhs_config.get_columns()

df_all, couple_all = dhs.get_all_data(RAW_DATA, range(1993, 2002))

100%|█████████████████████████████████████████████| 9/9 [01:10<00:00,  7.84s/it]


In [11]:
df_all.to_csv(os.path.join(TRS_DATA, 'df_all.csv'))

In [12]:
df_all[column_variables + column_others].to_csv(os.path.join(TRS_DATA, 'df_reduce.csv'))

In [13]:
couple_all.to_csv(os.path.join(TRS_DATA, 'couple_all.csv'))